In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

从2024年9月15日开始，您只能通过转移到Vertex AI Gemini提示和调整来定制分类、实体抽取和情感分析模型。不再提供为Vertex AI AutoML进行文本分类、实体抽取和情感分析目标进行训练或更新模型。您可以在2025年6月15日之前继续使用现有的Vertex AI AutoML文本目标。有关Gemini如何通过改进的提示功能提供增强用户体验的更多信息，请参阅 [调整介绍](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-gemini-overview)。

用于批量预测的AutoML训练文本情感分析模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl_text_sentiment_analysis_batch_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl_text_sentiment_analysis_batch_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/automl_text_sentiment_analysis_batch_prediction.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何使用Vertex AI SDK创建文本情感分析模型，并使用Google Cloud的AutoML模型进行批量预测。您可以在[这里](https://cloud.google.com/vertex-ai/docs/start/automl-users)找到更多信息。

目标

在本教程中，您将从一个Python脚本中创建一个AutoML文本情感分析模型，然后使用Vertex SDK进行批量预测。您也可以选择使用`gcloud`命令行工具或在线使用Cloud Console来创建和部署模型。

完成的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行按需预测，并实时返回结果。

* 批量预测服务：对整个实例集进行排队（批量）预测，后台存储结果，并在准备好时将结果存储在Cloud Storage桶中。

### 数据集

本教程使用的数据集是来自[data.world 数据集](https://data.world)的[Crowdflower Claritin-Twitter 数据集](https://data.world/crowdflower/claritin-twitter)。您在本教程中使用的数据集版本存储在一个公共云存储桶中。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您预计的使用量生成成本估算。

### 安装

安装以下软件包以执行此笔记本。

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform -q

! pip3 install --upgrade tensorflow -q

仅限协作：取消以下单元格的注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 开始之前

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
- 运行`gcloud config list`
- 运行`gcloud projects list`
- 查看支持页面：[定位项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

您还可以更改 Vertex AI 使用的 “REGION” 变量。
了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动进行认证。请按照以下相关说明操作。

**1. Vertex AI Workbench** 
- 无需进行任何操作，因为您已经认证过了。

**2. 本地 JupyterLab 实例**，取消注释并运行。

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
- 在这里查看所有的认证选项：[Google Cloud Platform Jupyter Notebook 认证指南](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_authentication_guide.ipynb)

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当你的存储桶尚不存在时：运行以下单元格以创建你的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。

### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# 教程

现在你已经准备好开始创建自己的AutoML文本情感分析模型。

云存储训练数据的位置。
现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/language/claritin.csv"
SENTIMENT_MAX = 4

快速查看您的数据

本教程使用存储在公共云存储桶中的Crowdflower Claritin-Twitter数据集的一个版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`TextDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`: `Dataset`资源的可读名称。
- `gcs_source`: 一个或多个数据集索引文件列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`: 数据项的数据标记模式。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.TextDataset.create(
    display_name="Crowdflower Claritin-Twitter",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.sentiment,
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行这个管道。

#### 创建训练管道

使用`AutoMLTextTrainingJob`类创建一个AutoML训练管道，使用以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：训练模型的任务类型。
  - `classification`：文本分类模型。
  - `sentiment`：文本情感分析模型。
  - `extraction`：文本实体提取模型。
- `multi_label`：如果是分类任务，是单标签（False）还是多标签（True）。
- `sentiment_max`：如果是情感分析任务，则是最大情感值。

实例化的对象是训练管道的DAG（有向无环图）。

In [ ]:
dag = aiplatform.AutoMLTextTrainingJob(
    display_name="claritin",
    prediction_type="sentiment",
    sentiment_max=SENTIMENT_MAX,
)

print(dag)

#### 运行训练流程

接下来，您可以通过调用`run`方法来运行DAG以开始训练作业，方法需要以下参数：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（保留数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。

完成`run`方法后，会返回`Model`资源。

训练流程的执行需要最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="claritin",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

## 检查模型评估分数

在您的模型训练完成后，您可以使用`list_model_evaluations()`方法查看其评估分数。该方法将返回每个评估切片的迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

发送批处理预测请求

向您的模型发送批处理预测。

获取测试项目

现在对您的 Vertex 模型进行批量预测。 您将使用数据集中的任意示例作为测试项目。 请不要担心这些示例可能已经用于训练模型 - 我们只是想演示如何进行预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2

if len(test_items[0]) == 4:
    _, test_item_1, test_label_1, _ = str(test_items[0]).split(",")
    _, test_item_2, test_label_2, _ = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1, _ = str(test_items[0]).split(",")
    test_item_2, test_label_2, _ = str(test_items[1]).split(",")


print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 创建批处理输入文件

现在创建一个批处理输入文件，将其存储在您的本地云存储桶中。批处理输入文件只能采用JSONL格式。对于JSONL文件，您对每个数据项（实例）都要每行制作一个字典条目。该字典包含键/值对：

- `content`：文本项文件的云存储路径。
- `mime_type`：内容类型。在我们的示例中，是一个`text`文件。

例如：

                        {'content': '[your-bucket]/file1.txt', 'mime_type': 'text'}

In [ ]:
import json

import tensorflow as tf

gcs_test_item_1 = BUCKET_URI + "/test1.txt"
with tf.io.gfile.GFile(gcs_test_item_1, "w") as f:
    f.write(test_item_1 + "\n")
gcs_test_item_2 = BUCKET_URI + "/test2.txt"
with tf.io.gfile.GFile(gcs_test_item_2, "w") as f:
    f.write(test_item_2 + "\n")

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": gcs_test_item_1, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")
    data = {"content": gcs_test_item_2, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 发起批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法来进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的人类可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `sync`：如果设置为True，则调用将在等待异步批处理作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="claritin",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

### 等待批量预测作业完成

接下来，请等待批量作业完成。或者，可以在 `batch_predict()` 方法中将参数 `sync` 设置为 `True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的云存储输出桶。您可以调用方法iter_outputs()来获取生成结果的每个云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `sentiment`：情绪。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
import os

delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

# Delete batch
batch_predict_job.delete()

# Delete model
model.delete()

# Delete text dataset
dataset.delete()

# Delete training job
dag.delete()